https://towardsdatascience.com/pyspark-data-manipulation-tutorial-8c62652f35fa

In [1]:
import sys, os
environment = ['PYSPARK_PYTHON', 'PYSPARK_DRIVER_PYTHON']
for var in environment:
    os.environ[var] = sys.executable

In [2]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/28 03:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()

In [10]:
rdd = session.sparkContext.parallelize([1,2,3])

In [11]:
rdd.take(num=2)

[1, 2]

In [12]:
rdd.count()

3

In [13]:
rdd.collect()

[1, 2, 3]

In [16]:
df = session.createDataFrame(
  [[1,2,3], [4,5,6]], ['column1', 'column2', 'column3']
)

In [17]:
df.show(n=3)

+-------+-------+-------+
|column1|column2|column3|
+-------+-------+-------+
|      1|      2|      3|
|      4|      5|      6|
+-------+-------+-------+



In [18]:
a = list(range(10))
a.append(11)

In [20]:
st = "my string"
st += " is pretty"

In [22]:
rdd.map(lambda x: x*100)

PythonRDD[15] at RDD at PythonRDD.scala:53

In [23]:
my_rdd = rdd.map(lambda x: x*100)

In [24]:
import pyspark.sql.functions as funcs
import pyspark.sql.types as types

def multiply_by_ten(number):
    return number*10.0
multiply_udf = funcs.udf(multiply_by_ten, types.DoubleType())
transformed_df = df.withColumn(
    'multiplied', multiply_udf('column1')
)
transformed_df.show()

+-------+-------+-------+----------+
|column1|column2|column3|multiplied|
+-------+-------+-------+----------+
|      1|      2|      3|      10.0|
|      4|      5|      6|      40.0|
+-------+-------+-------+----------+



In [31]:
import math
import pyspark.sql.types as types
def take_log_in_all_columns(row: types.Row):
    old_row = row.asDict()
    new_row = {f'log({column_name})': math.log(value) 
    for column_name, value in old_row.items()}
    return types.Row(**new_row)
    
logarithmic_dataframe = df.rdd.map(take_log_in_all_columns).toDF()

In [33]:
df.select('column1', 'column2')

DataFrame[column1: bigint, column2: bigint]

In [34]:
df.where('column1 = 3')

DataFrame[column1: bigint, column2: bigint, column3: bigint]

In [36]:
df.join(df, ['column1'], how='inner')

DataFrame[column1: bigint, column2: bigint, column3: bigint, column2: bigint, column3: bigint]

In [38]:
df.createOrReplaceTempView('table1')

In [39]:
df2 = session.sql("SELECT column1 AS f1, column2 as f2 from table1")

In [40]:
df3 = df.withColumn(
    'derived_column', df['column1'] + df['column2'] * df['column3']
)

In [41]:
ADULT_COLUMN_NAMES = [
     "age",
     "workclass",
     "fnlwgt",
     "education",
     "education_num",
     "marital_status",
     "occupation",
     "relationship",
     "race",
     "sex",
     "capital_gain",
     "capital_loss",
     "hours_per_week",
     "native_country",
     "income"
 ]

In [57]:
csv_df = session.read.csv(
     'adult.data', header=False, inferSchema=True)
for new_col, old_col in zip(ADULT_COLUMN_NAMES, csv_df.columns):
     csv_df = csv_df.withColumnRenamed(old_col, new_col)

In [58]:
csv_df.head()

Row(age=39, workclass=' State-gov', fnlwgt=77516.0, education=' Bachelors', education_num=13.0, marital_status=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', sex=' Male', capital_gain=2174.0, capital_loss=0.0, hours_per_week=40.0, native_country=' United-States', income=' <=50K')

In [59]:
csv_df.describe().show()

+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|summary|               age|   workclass|            fnlwgt|    education|    education_num|marital_status|       occupation|relationship|               race|    sex|      capital_gain|    capital_loss|    hours_per_week|native_country|income|
+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|  count|             32561|       32561|             32561|        32561|            32561|         32561|            32561|       32561|              32561|  32561|             32561|           32561|             32561|         32561| 32561|
|   mean| 38.58164675532

In [60]:
work_hours_df = csv_df.groupBy(
    'age'
).agg(
    funcs.avg('hours_per_week'),
    funcs.stddev_samp('hours_per_week')
).sort('age')

In [61]:
url = f"jdbc:postgresql://your_host_ip:5432/your_database"
properties = {'user': 'your_user', 'password': 'your_password'}
# read from a table into a dataframe
df = session.read.jdbc(
    url=url, table='your_table_name', properties=properties
)

Py4JJavaError: An error occurred while calling o459.jdbc.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:33)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:350)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:274)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:245)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader

In [ ]:
transformed_df.write.jdbc(
    url=url, table='new_table', mode='append', properties=properties
)